<div style="border:solid  green  2px; padding: 20px"> <h1 style="color: green">Проект по SQL</h1>

Цель - определить основные тенденции в сегменте выпуска книг для формирования нового предложения.

In [2]:
import pandas as pd
from sqlalchemy import create_engine 
from pandas.io.sql import read_sql

db_config = {'user': 'praktikum_student', 
             'pwd': 'Sdf4$2;d-d30pp', 
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,
             'db': 'data-analyst-final-project-db'} 

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                         db_config['pwd'], 
                                                         db_config['host'], 
                                                         db_config['port'],
                                                         db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
query = 'select * from books limit 5'
read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = 'select * from authors limit 5'
read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = 'select * from publishers limit 5'
read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = 'select * from ratings limit 5'
read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = 'select * from reviews limit 5'
read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Количество книг вышедших после 1 января 2000 года

In [8]:
query = '''
    select count(*) 
        from books 
        where publication_date > \'2000-01-01\'
    '''

read_sql(query, con = engine) 

,count
0,819


Большую часть датасета составляют книги, вышедшие в 21 веке 819 из 1000. Скорее всего современная литература пользуется большей популярностью.

### Считаем количество обзоров на книгу

In [17]:

query = '''select books.book_id, books.title, 
                count(distinct(reviews.review_id)) as reviews_count, 
                round(avg(ratings.rating), 2) as average_rating
            from books 
            left join ratings 
                on ratings.book_id = books.book_id 
            left join reviews
                on reviews.book_id = books.book_id
            group by books.book_id, books.title
            order by reviews_count desc
            limit 10'''

read_sql(query, con = engine) 

,book_id,title,reviews_count,average_rating
0,948,Twilight (Twilight #1),7,3.66
1,696,The Da Vinci Code (Robert Langdon #2),6,3.83
2,207,Eat Pray Love,6,3.40
3,627,The Alchemist,6,3.79
4,673,The Catcher in the Rye,6,3.83
5,695,The Curious Incident of the Dog in the Night-Time,6,4.08
6,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
7,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
8,497,Outlander (Outlander #1),6,4.13
9,656,The Book Thief,6,4.26


Большее количество отзывов получают экранизованные книги, что логично. При этом колчество отзывов не влияет на рейтинг произведения.

### Издательство, которое выпустило больше всего книг толщиной больше 50 страниц


In [10]:

query = '''
    select count(*) as num_books, publisher     
    from books 
    left join publishers on publishers.publisher_id = books.publisher_id
    where num_pages > 50 
    group by publishers.publisher_id, publishers.publisher 
    order by num_books desc
    limit 1
    '''

read_sql(query, con = engine) 

,num_books,publisher
0,42,Penguin Books


Это Пингвин Букс.

### Автор с самой высокой средней оценкой, при учете авторов с > 50 оценками

In [11]:

query = '''
select 
        count(ratings.rating_id) as num_ratings, 
        round(avg(ratings.rating), 2) as avg_rating,
        authors.author as author_name
        from books
    join authors on authors.author_id = books.author_id
    join ratings on ratings.book_id = books.book_id
    where books.author_id in (
            select author_id from books 
            join ratings on ratings.book_id = books.book_id
            group by books.book_id
            having count(ratings.rating_id) >= 50
        )    
    group by authors.author_id, authors.author
    order by avg_rating desc
    limit 10
    '''

read_sql(query, con = engine) 

,num_ratings,avg_rating,author_name
0,312,4.29,J.K. Rowling/Mary GrandPré
1,53,4.26,Markus Zusak/Cao Xuân Việt Khương
2,166,4.24,J.R.R. Tolkien
3,54,4.20,Louisa May Alcott
4,84,4.13,Rick Riordan
5,71,3.90,William Golding
6,98,3.85,J.D. Salinger
7,57,3.79,Paulo Coelho/Alan R. Clarke/Özdemir İnce
8,66,3.79,William Shakespeare/Paul Werstine/Barbara A. M...
9,65,3.74,Lois Lowry


Это Дж. Роулинг

### Среднее количество обзоров от пользователей, поставивших больше 50 оценок

In [13]:

query = '''select avg(num_reviews) from (
    select count(*) as num_reviews from reviews where username in 
    (
        select username 
        from ratings 
        group by username
        having count(*) > 50
    )
    group by username) as reviews
    '''

read_sql(query, con = engine) 

,avg
0,24.333333


В среднем, пользователи, оставляющие большое количество оценок, так же активны и в написании отзывов - в среднем 24.